<a href="https://colab.research.google.com/github/ucfilho/ANN/blob/master/ANN_2022/Image_07/class_007_image_processing_blood_cells_004.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
tf.__version__

'2.9.2'

In [2]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, BatchNormalization, ZeroPadding2D, MaxPool2D, Activation, add
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

import cv2
import numpy as np

from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

from glob import glob

import zipfile
import pandas as pd


from sklearn.metrics import classification_report, confusion_matrix

In [3]:
IMAGE_SIZE = [224,224]
ImgSize = IMAGE_SIZE[0]
epochs = 16
batch_size = 128

In [4]:
!gdown 1CU4UkqJae07u1tKfp2rajWwwZA0SKiIm

Downloading...
From: https://drive.google.com/uc?id=1CU4UkqJae07u1tKfp2rajWwwZA0SKiIm
To: /content/blood_cell_images.zip
100% 110M/110M [00:00<00:00, 210MB/s]


In [5]:
FILE='blood_cell_images.zip'

In [6]:
archive = zipfile.ZipFile(FILE, 'r')
archive.extractall()

In [7]:
!ls

blood_cell_images  blood_cell_images.zip  __MACOSX  sample_data


In [8]:
train_path = 'blood_cell_images/TRAIN'
test_path = 'blood_cell_images/TEST'
images_test = glob(test_path + '/*/*.jp*g')
images_train = glob(train_path + '/*/*.jp*g')

In [9]:
folders_train = glob(train_path + '/*')
folders_test = glob(test_path + '/*')

In [10]:
folders_train

['blood_cell_images/TRAIN/LYMPHOCYTE',
 'blood_cell_images/TRAIN/MONOCYTE',
 'blood_cell_images/TRAIN/NEUTROPHIL',
 'blood_cell_images/TRAIN/EOSINOPHIL']

In [11]:
folders_test

['blood_cell_images/TEST/LYMPHOCYTE',
 'blood_cell_images/TEST/MONOCYTE',
 'blood_cell_images/TEST/NEUTROPHIL',
 'blood_cell_images/TEST/EOSINOPHIL']

In [12]:
ntrain, ntest = len(images_train),len(images_test)
ntrain,ntest

(9957, 2487)

In [13]:
images_train[:4]

['blood_cell_images/TRAIN/LYMPHOCYTE/_18_6064.jpeg',
 'blood_cell_images/TRAIN/LYMPHOCYTE/_14_2074.jpeg',
 'blood_cell_images/TRAIN/LYMPHOCYTE/_0_6532.jpeg',
 'blood_cell_images/TRAIN/LYMPHOCYTE/_22_1298.jpeg']

In [14]:
types = ['NEUTROPHIL', 'EOSINOPHIL', 'MONOCYTE', 'LYMPHOCYTE']

In [15]:

def load_images(images,types=types):

  nsize = len(images)
  X =[]
  y =[]
  names =[]
  k = 0

  for i in range(nsize): 
    img = image.load_img(images[i])
    im = cv2.resize(np.array(img), (224, 224))
    im = preprocess_input(im)
    k += 1
    if( k % 3 == 0):
      X.append(im)

      if(types[0] in images[i]):
        names.append(types[0])
        y.append(0)
      elif(types[1] in images[i]):
        names.append(types[1])
        y.append(1)
      elif(types[2] in images[i]):
        names.append(types[2])
        y.append(2)
      else:
        names.append(types[3])
        y.append(3)

  X =np.array(X)
  return X,y,names

In [16]:

X_train, y_train, names_train = load_images(images_train)
X_test, y_test, names_test = load_images(images_test)

In [17]:
X_train[0].shape

(224, 224, 3)

In [18]:
df = pd.DataFrame({'name':names_train,'class':y_train})
y_train = pd.get_dummies(df['class'])
y_train

,0,1,2,3
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
3314,0,1,0,0
3315,0,1,0,0
3316,0,1,0,0
3317,0,1,0,0


In [19]:
for i in range(4): 
  df_plot= df[(df["class"] == i)]
  n= df_plot.shape[0]
  print(types[i],n)

NEUTROPHIL 833
EOSINOPHIL 833
MONOCYTE 826
LYMPHOCYTE 827


In [20]:
df = pd.DataFrame({'name':names_test,'class':y_test})
y_test = pd.get_dummies(df['class'])
y_test

,0,1,2,3
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
824,0,1,0,0
825,0,1,0,0
826,0,1,0,0
827,0,1,0,0


In [21]:
#X_train = np.array(X_train)
#X_test = np.array(X_test)
#X_train = X_train.reshape(-1, ImgSize, ImgSize, 3) / 255.0
#X_test = X_test.reshape(-1, ImgSize, ImgSize, 3) / 255.0

In [22]:
from tensorflow.keras.applications import vgg16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,  Flatten, Dropout
# Init the VGG model 
vgg_conv = vgg16.VGG16(weights='imagenet') #, include_top=False, input_shape=(Img_Size, Img_Size, 3))

# Freeze all the layers
for layer in vgg_conv.layers[:]:
    layer.trainable = False

# Create the model
model = Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
D = len(types)
# Add new layers
#model.add(Flatten())
#model.add(Dense(300, activation='relu')) #relu
#model.add(Dropout(0.2))
#model.add(Dense(100, activation='relu')) #relu
#model.add(Dropout(0.2))
model.add(Dense(D, activation= 'softmax'))#'relu'))

In [23]:
'''model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)'''

"model.compile(\n  loss='categorical_crossentropy',\n  optimizer='adam',\n  metrics=['accuracy']\n)"

In [24]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)
r = model.fit(x=X_train, 
              y=y_train, 
              validation_data=(X_test,y_test),
              epochs=200, 
              verbose=2)
print("Returned:", r)

Epoch 1/200
104/104 - 30s - loss: 1.3861 - accuracy: 0.2510 - val_loss: 1.3862 - val_accuracy: 0.2497 - 30s/epoch - 293ms/step
Epoch 2/200
104/104 - 22s - loss: 1.3861 - accuracy: 0.2648 - val_loss: 1.3861 - val_accuracy: 0.2545 - 22s/epoch - 211ms/step
Epoch 3/200
104/104 - 21s - loss: 1.3860 - accuracy: 0.2612 - val_loss: 1.3861 - val_accuracy: 0.2666 - 21s/epoch - 204ms/step
Epoch 4/200
104/104 - 21s - loss: 1.3860 - accuracy: 0.2712 - val_loss: 1.3861 - val_accuracy: 0.2799 - 21s/epoch - 201ms/step
Epoch 5/200
104/104 - 21s - loss: 1.3859 - accuracy: 0.3378 - val_loss: 1.3860 - val_accuracy: 0.2726 - 21s/epoch - 202ms/step
Epoch 6/200
104/104 - 21s - loss: 1.3858 - accuracy: 0.3007 - val_loss: 1.3860 - val_accuracy: 0.2907 - 21s/epoch - 203ms/step
Epoch 7/200
104/104 - 21s - loss: 1.3858 - accuracy: 0.3459 - val_loss: 1.3859 - val_accuracy: 0.3052 - 21s/epoch - 203ms/step
Epoch 8/200
104/104 - 21s - loss: 1.3857 - accuracy: 0.3378 - val_loss: 1.3859 - val_accuracy: 0.3293 - 21s/epo

In [ ]:
#X_train.shape

In [ ]:
'''
r = model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    verbose=2
)'''

In [ ]:
def retorna(y):
  resp = []
  for yi in y:
    vmax =max(yi)
    for idx, x in enumerate(yi):
      if(x == vmax):
        resp.append(idx)
  return resp

In [ ]:
plt.plot(r.history['accuracy'],label='accuracy')
#plt.plot(r.history['val_accuracy'],label='val_accuracy')
#plt.legend()

In [ ]:
predict=model.predict(X_test)

In [ ]:
y_calc = retorna(predict)
y_test = retorna(np.array(y_test))

In [ ]:
confusion_matrix(y_test, y_calc)

In [ ]:
# Classification Report
print('\n\n', classification_report(y_test, y_calc, target_names=types))